# 5장, 판다스 시작하기


In [1]:
!pip install -Uqq numpy pandas
# !pip install -Uqq datasets # in HuggingFace

In [2]:
import numpy as np
import pandas as pd
# import torch

from pandas import Series, DataFrame

## 5.1 pandas 데이터 구조 소개
### 시리즈


### 데이터 프레임

In [4]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [10]:
frame.head(4) # frame.tail()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4


In [ ]:
pd.DataFrame(data, columns=["year", "state", "pop"]) # 컬럼의 순서를 커스터마이즈..

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


In [13]:
# 열 추출
frame['state']  # == frame.state

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [15]:
#frame[2]
# 이렇게는 사용 할 수 없음.

### 인덱스 객체



## 5.2 필수 기능

### 재색인화
### 축에서 항목 삭제


### 인덱싱, 선택 및 필터링

#### Series 인덱싱

In [13]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])
obj1,obj2

(2    1
 0    2
 1    3
 dtype: int64,
 a    1
 b    2
 c    3
 dtype: int64)

In [15]:
obj2['a':'b']

a    1
b    2
dtype: int64

#### DataFrame 인덱싱

기본적으로는 컬럼 인덱싱이다.

In [4]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
   index=["Ohio", "Colorado", "Utah", "New York"],
   columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
# 두 개의 axis 가 있다고 보면, 한 axis 를 인덱싱 하는 것인데 기본적으로 컬럼 인덱싱을 한다는 것이 좀 의외이긴 하다.

data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [20]:
data[['two', 'three']]

,two,three
Ohio,1,2
Colorado,5,6
Utah,9,10
New York,13,14


In [ ]:
# 이건 일반적이지 않은 특수 사용 예시이다. 너무 확장 해석을 하지는 말자.
# 1. 슬라이싱으로 column 이 아닌 row 를 추출한다.
#
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [23]:
# bool 수식으로 인덱싱 하는 것도 특수 사용 예시.
data[ data['three'] > 7 ]

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [24]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [ ]:
# loc 을 사용해야 제대로 된 2차 배열 처럼 간주된다.
#
data.loc["Colorado"]
#
# 결과는 Series 인데 세로로 표시되어서 좀 이상하게 보이긴 함.

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int64

In [26]:
data.loc[["Colorado", "New York"]]

,one,two,three,four
Colorado,4,5,6,7
New York,12,13,14,15


In [ ]:
# 행, 열을 모두 인덱싱. 콤마를 추가하여 두 개의 인덱스를 제공.

data.loc["Colorado", "two"]

np.int64(5)

In [29]:
# 둘 중 리스트(시퀀스) 가 포함되면 차원이 줄어들 지 않음.
data.loc["Colorado", ["two", "three"]]

two      5
three    6
Name: Colorado, dtype: int64

In [ ]:
# 값은 하나 뿐이지만, 차원을 줄어들지 않아서 여전히 DataFrame 이다.
data.loc[["Colorado"], ["two"]]

,two
Colorado,5


In [ ]:
# 여기에서 슬라이스 사용 가능. 반드시 loc, iloc 에서만 쓸 수 있다.
data.loc[:'Colorado', 'two':]

,two,three,four
Ohio,1,2,3
Colorado,5,6,7


In [ ]:
# 이런 직접 access 는 사용하지 않는 것이 좋다.
try:
    data['Colorado']
except Exception as e:
    print(e.__class__, e)

<class 'KeyError'> 'Colorado'


- 결론
  - 항상 `.loc[]`, `.iloc[]` 을 활용하자!


#### 체인 인덱싱 문제

In [ ]:
print(data)

# 이런 식의 체인 인덱싱은 좋지 않다!  경고가 발생해서 문제를 알려주긴 하지만...
data.loc[data.three == 6]["three"] = -1
data

          one  two  three  four
Ohio        0    1      2     3
Colorado    4    5      6     7
Utah        8    9     10    11
New York   12   13     14    15


/var/folders/gk/5yll9c4s4574ff6wr892k56r0000gn/T/ipykernel_77523/744742152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.three == 6]["three"] = -1


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
# 한번의 인덱싱으로 끝내야 한다!
#
data.loc[data.three == 6, "three"] = -1
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,-1,7
Utah,8,9,10,11
New York,12,13,14,15


### 산술 및 데이터 정렬
### 함수 적용 및 매핑
### 정렬 및 순위
### 중복 레이블이 있는 축 인덱스


## 5.3 기술 통계 요약 및 계산
